# Set Up

In [2]:
import pandas as pd
from pandas_schema import Column, Schema
from pandas_schema.validation import (
    DateFormatValidation,
    LeadingWhitespaceValidation,
    TrailingWhitespaceValidation,
    CanConvertValidation,
    MatchesPatternValidation,
    InRangeValidation,
    InListValidation,
    _SeriesValidation,
    CustomSeriesValidation,
    CustomElementValidation,
)

from io import StringIO

In [ ]:
#file_path = r"C:\Users\tape\Documents\GitHub\Pandas\SampleFiles\employees.csv"
#df = pd.read_csv(file_path, dtype="object")

# Example Column Validations

In [89]:
test_data = pd.DataFrame({
    'col1': [
        '',
        '13',
        'a',
        '8',
        'the',
    ],
    'col2': [
        '123',
        '13.50',
        123,
        123.5,
        'the',
    ],
    'col3': [
        '1000',
        '1300.50',
        123,
        123.5,
        999,
    ]
})
test_data

,col1,col2,col3
0,,123,1000
1,13,13.50,1300.50
2,a,123,123
3,8,123.5,123.5
4,the,the,999


In [91]:
schema = Schema([
    Column('col1', [CanConvertValidation(int)], allow_empty=False),
    Column('col2', [CanConvertValidation(int, message="is not a number")]), # can do custom message
])

errors = schema.validate(test_data[['col1','col2']])

for error in errors:
    print(f"{error.column} value of'{error.value}' {error.message};")

col1 value of'' cannot be converted to type <class 'int'>;
col2 value of'13.50' is not a number;
col1 value of'a' cannot be converted to type <class 'int'>;
col1 value of'the' cannot be converted to type <class 'int'>;
col2 value of'the' is not a number;


# More Examples

In [74]:
test_data = pd.read_csv(StringIO('''Given Name,Family Name,Age,Sex,Customer ID
Gerald ,Hampton,82,Male,2582GABK
Yuuwa,Miyake,270,male,7951WVLW
Edyta,Majewska ,50,Female,775ANSID
'''))
test_data

,Given Name,Family Name,Age,Sex,Customer ID
0,Gerald,Hampton,82,Male,2582GABK
1,Yuuwa,Miyake,270,male,7951WVLW
2,Edyta,Majewska,50,Female,775ANSID


In [13]:
schema = Schema([
    Column('Given Name', [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
    Column('Family Name', [LeadingWhitespaceValidation(), TrailingWhitespaceValidation()]),
    Column('Age', [CanConvertValidation(int), InRangeValidation(0, 120)]),
    Column('Sex', [InListValidation(['Male', 'Female', 'Other'])]),
    Column('Customer ID', [MatchesPatternValidation(r'\d{4}[A-Z]{4}')]),
])

errors = schema.validate(test_data)

for error in errors:
    print(error)

{row: 0, column: "Given Name"}: "Gerald " contains trailing whitespace
{row: 1, column: "Age"}: "270" was not in the range [0, 120)
{row: 1, column: "Sex"}: "male" is not in the list of legal options (Male, Female, Other)
{row: 2, column: "Family Name"}: "Majewska " contains trailing whitespace
{row: 2, column: "Customer ID"}: "775ANSID" does not match the pattern "\d{4}[A-Z]{4}"


# Custom Class Validation Example

In [75]:
class NoMoreMoneyThan(_SeriesValidation):
    
    def __init__(self, amount_limit, **kwargs):
        super().__init__(**kwargs)
        self.amount_limit = int(amount_limit)

    @property
    def default_message(self):
        return f"is higher than {self.amount_limit}"
        
    def validation_function(self, var):
        try:
            return int(var) <= self.amount_limit
        except:
            return False

    def validate(self, series: pd.Series) -> pd.Series:
        return series.apply(self.validation_function)

In [76]:
test_data = pd.DataFrame({
    'col1': [
        'an',
        '13',
        'a',
        '8',
        'the',
    ],
    'col2': [
        '123',
        '13.50',
        123,
        123.5,
        'the',
    ],
    'col3': [
        '1000',
        '1300.50',
        123,
        123654,
        999,
    ]
})
test_data

,col1,col2,col3
0,an,123,1000
1,13,13.50,1300.50
2,a,123,123
3,8,123.5,123654
4,the,the,999


In [77]:
schema = Schema([
    Column('col1', [CanConvertValidation(str)]),
    Column('col2', [CanConvertValidation(int, message="is not a number")]),
    Column('col3', [NoMoreMoneyThan(1000)]),
])

errors = schema.validate(test_data)

for error in errors:
    print(f"{error.column} value of'{error.value}' {error.message};")

col2 value of'13.50' is not a number;
col3 value of'1300.50' is higher than 1000;
col3 value of'123654' is higher than 1000;
col2 value of'the' is not a number;


In [78]:
schema = Schema([
    Column('col1', [CanConvertValidation(str)]),
    Column('col2', [CanConvertValidation(int, message="is not a number")]),
    Column('col3', [CanConvertValidation(int), InRangeValidation(0,1000)]), # , & |
])

errors = schema.validate(test_data)

for error in errors:
    print(f"{error.column} value of'{error.value}' {error.message};")

col3 value of'1000' was not in the range [0, 1000);
col2 value of'13.50' is not a number;
col3 value of'1300.50' cannot be converted to type <class 'int'>;
col3 value of'1300.50' was not in the range [0, 1000);
col3 value of'123654' was not in the range [0, 1000);
col2 value of'the' is not a number;


# Custom Type Example

In [79]:
from decimal import Decimal

schema = Schema([
    Column('col1', [CanConvertValidation(Decimal)]),
    Column('col2', [CanConvertValidation(int)]),
])

errors = schema.validate(test_data[['col1','col2']])

for error in errors:
    print(f"{error.column} value of'{error.value}' {error.message};")

col1 value of'an' cannot be converted to type <class 'decimal.Decimal'>;
col2 value of'13.50' cannot be converted to type <class 'int'>;
col1 value of'a' cannot be converted to type <class 'decimal.Decimal'>;
col1 value of'the' cannot be converted to type <class 'decimal.Decimal'>;
col2 value of'the' cannot be converted to type <class 'int'>;


# Join Error Messages

In [32]:
from collections import defaultdict
grouped_errors = defaultdict(list)
for error in errors:
    grouped_errors[error.row].append(error)
        
for row in grouped_errors:
    error_messages = map(lambda e: f"{e.column} {e.message}", grouped_errors[row])
    test_data.at[row, "errorCode"] = ";".join(str(x) for x in error_messages)
        
test_data

,col1,col2,col3,errorCode
0,an,123,1000,col1 cannot be converted to type <class 'decim...
1,13,13.50,1300.50,col2 cannot be converted to type <class 'int'>
2,a,123,123,col1 cannot be converted to type <class 'decim...
3,8,123.5,123654,NaN
4,the,the,999,col1 cannot be converted to type <class 'decim...


# MultiIndex Example

In [80]:
df = pd.DataFrame({
    ("ScheduleAmount"): [150, 2, 3, "a"],
    ("ScheduleFrequency"): ["a", "b", "c", ""],
    ("ScheduleStart"): ["20200101","", "20200101","A"],
})
df["AllocationScheduleFields"] = pd.MultiIndex.from_frame(df, names=['amount','freq','start'])
df

,ScheduleAmount,ScheduleFrequency,ScheduleStart,AllocationScheduleFields
0,150,a,20200101,"(150, a, 20200101)"
1,2,b,,"(2, b, )"
2,3,c,20200101,"(3, c, 20200101)"
3,a,,A,"(a, , A)"


In [81]:
class AllOrNone(_SeriesValidation):
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    @property
    def default_message(self):
        return 'were partially populated. Provide all 3 or none.'
        
    def validation_function(self, var):
        try:
            return var.count("") == 0 or var.count("") == 3
        except:
            return False

    def validate(self, series: pd.Series) -> pd.Series:
        return series.apply(self.validation_function)

In [82]:
schema = Schema([
    Column('AllocationScheduleFields', [AllOrNone()]),
    Column('ScheduleAmount', [InRangeValidation(0, 1000000), CanConvertValidation(int)]),
    Column('ScheduleFrequency', [InListValidation(["A","M","a"])]),
    Column('ScheduleStart', [DateFormatValidation("%Y%m%d")]),
])


errors = schema.validate(df)

for error in errors:
    #print(error)
    #print(f"{error.column} value of'{error.value}' {error.message};")
    print(f"{error.column} {error.message};")

AllocationScheduleFields were partially populated. Provide all 3 or none.;
ScheduleFrequency is not in the list of legal options (A, M, a);
ScheduleStart does not match the date format string "%Y%m%d";
ScheduleFrequency is not in the list of legal options (A, M, a);
AllocationScheduleFields were partially populated. Provide all 3 or none.;
ScheduleAmount was not in the range [0, 1000000);
ScheduleAmount cannot be converted to type <class 'int'>;
ScheduleFrequency is not in the list of legal options (A, M, a);
ScheduleStart does not match the date format string "%Y%m%d";


## different way to make a combined column

In [66]:
df = pd.DataFrame({
    ("ScheduleAmount"): [150, 2, 3, "a"],
    ("ScheduleFrequency"): ["a", "b", "c", ""],
    ("ScheduleStart"): ["20200101","", "20200101","A"],
})
df.insert(loc=0, column='AllocationScheduleFields', value=list(zip(df.ScheduleAmount, df.ScheduleFrequency, df.ScheduleStart)), allow_duplicates=False)
df

,AllocationScheduleFields,ScheduleAmount,ScheduleFrequency,ScheduleStart
0,"(150, a, 20200101)",150,a,20200101
1,"(2, b, )",2,b,
2,"(3, c, 20200101)",3,c,20200101
3,"(a, , A)",a,,A


# Column/Row Validator

In [47]:
df = pd.DataFrame({
    ("ScheduleAmount"): [150, 2, 3, "a",""],
    ("ScheduleFrequency"): ["a", "b", "c", "",""],
    ("ScheduleStart"): ["20200101","", "20200101","A",""],
})
df

,ScheduleAmount,ScheduleFrequency,ScheduleStart
0,150,a,20200101
1,2,b,
2,3,c,20200101
3,a,,A
4,,,


In [4]:
df.columns

Index(['ScheduleAmount', 'ScheduleFrequency', 'ScheduleStart'], dtype='object')

In [48]:
column_list = ['ScheduleAmount', 'ScheduleFrequency', 'ScheduleStart']

def row_validation(row):
    result = []
    for col in column_list:
        result.append(row[col] == "")
    return all(x == result[0] for x in result)

    
error_series = df.apply(lambda row: row_validation(row), axis = 1)
error_series

0     True
1    False
2     True
3    False
4     True
dtype: bool

In [78]:
for row_number, row in df.iterrows():
    row, row_number, df
    #print(row)
    print(row_validation(row))
    
    if row_validation(row):
        pass
    else:
        print("ValidationWarning")

        

False
ValidationWarning
False
ValidationWarning
True
False
ValidationWarning
True


In [74]:
column_dict = {'ScheduleAmount': "123", 'ScheduleFrequency': "", 'ScheduleStart': "20200101"}
for key, val in column_dict.items():
    df.at[0, key] = val
    
df

,ScheduleAmount,ScheduleFrequency,ScheduleStart
0,123,,20200101
1,2,b,
2,3,c,20200101
3,a,,A
4,,,


In [73]:
df

,ScheduleAmount,ScheduleFrequency,ScheduleStart
0,123,A,20200101
1,2,b,
2,3,c,20200101
3,a,,A
4,,,


In [11]:
df[empty_mask | populated_mask]

,ScheduleAmount,ScheduleFrequency,ScheduleStart
0,150,a,20200101
1,2,b,
2,3,c,20200101
3,a,,A


# Primay and Secondary Validations?

In [64]:
test_data = pd.DataFrame({
    'col1': [
        '',
        '1300.50',
        '123',
        '123654',
        None,
    ]
}, dtype='object')
test_data

,col1
0,
1,1300.50
2,123
3,123654
4,None


In [151]:
schema = Schema([
    Column('col1', [CanConvertValidation(int), InRangeValidation(0,1000)],allow_empty=True), # , & |
])

errors = schema.validate(test_data[['col1']])

for error in errors:
    print(f"{error.column} value of '{error.value}' {error.message};")

col1 value of '1300.50' cannot be converted to type <class 'int'>;
col1 value of '1300.50' was not in the range [0, 1000);
col1 value of '123654' was not in the range [0, 1000);


In [65]:
test_data['col1'].str.match("^$")

0     True
1    False
2    False
3    False
4     None
Name: col1, dtype: object

In [60]:
test_data['col1'].astype('str').str.match("^$")

0     True
1    False
2    False
3    False
4    False
Name: col1, dtype: bool

In [61]:
rules_list = "required"
rules_list = "required".split("|")
required = not "required" in rules_list

schema = Schema([
    #Column('col1', [~CanConvertValidation(Decimal) | ~InRangeValidation(0,1000)]), # , & |
    Column('col1', [CustomSeriesValidation(lambda s: ~s.astype('str').str.match("^$"), "is a required field.")],allow_empty=False),
    #Column('col1', [CustomElementValidation(lambda s: s != "", "is a required field.")]),
    
    
])

errors = schema.validate(test_data[['col1']])

for error in errors:
    print(f"{error.column} (value of '{error.value}') {error.message};")

col1 (value of '') is a required field.;


:param allow_empty: True if an empty column is considered valid. False if we leave that logic up to the Validation
allow_empty=False,

In [167]:
rules_list

['required']

In [136]:
rules_list = ["required"]
rules_list = "required".split("|")
not "required" in rules_list

False

In [168]:
rules_list = "required|date".split("|")
not "required" in rules_list

False

In [137]:
rules_list = "date".split("|")
not "required" in rules_list

True

In [176]:
RULES = {"required": [CustomSeriesValidation],"date": [CustomSeriesValidation]}
items = rules_list
[
        rule
        for item in items
        #if item.lower() not in SKIP
        for rule in RULES[item.lower()]
    ]

[pandas_schema.validation.CustomSeriesValidation,
 pandas_schema.validation.CustomSeriesValidation]

In [28]:
dict_test =  {
            "dependentHraAllocationScheduleAmount": "123",
            "dependentHraAllocationScheduleFrequencyCode": "A",
            "dependentHraAllocationScheduleStartDate": "20201231",
        }
key_list = list(dict_test.keys())
key_list

['dependentHraAllocationScheduleAmount',
 'dependentHraAllocationScheduleFrequencyCode',
 'dependentHraAllocationScheduleStartDate']

In [19]:
value = None
result = []

result.append(value) if value == "" else None
print(len(result))
result

0


[]

In [25]:
value = NoneType()
result.append(value)
result

NameError: name 'NoneType' is not defined

# File Date

In [11]:
test_data = pd.read_csv(StringIO("""clientOriginalFileName,systemAssignedBaseFileName,originalFileProcessingStartedAtTime,socialSecurityNumber,firstName,middleName,lastName,dateOfBirth,genderCode,eligibilityStartDate,eligibleIndicator,clientSpecificPlanId,clientSpecificDentalId,medicarePartAStartDate,medicarePartAEndDate,medicarePartBStartDate,medicarePartBEndDate,healthInsuranceClaimNumber,eventCode,dependentSocialSecurityNumber,dependentFirstName,dependentMiddleName,dependentLastName,dependentDateOfBirth,dependentGenderCode,dependentEligibilityStartDate,dependentEligibleIndicator,dependentClientSpecificPlanId,dependentClientSpecificDentalId,dependentMedicarePartAStartDate,dependentMedicarePartAEndDate,dependentMedicarePartBStartDate,dependentMedicarePartBEndDate,dependentHealthInsuranceClaimNumber,dependentEventCode,phoneNumber1,phoneNumber2,emailAddress,dependentEmailAddress,addressLine1,addressLine2,addressLine3,cityName,stateCode,zipCode,countryCode,countyName,campaignSegmentGuid,clientMemberId,hraAllocationScheduleAmount,hraAllocationScheduleFrequencyCode,hraAllocationScheduleStartDate,hasEndStageRenalDisease,dependentClientMemberId,dependentHraAllocationScheduleAmount,dependentHraAllocationScheduleFrequencyCode,dependentHraAllocationScheduleStartDate,dependentHasEndStageRenalDisease,reportingValue1,reportingValue2,reportingValue3,reportingValue4,reportingValue5,eligibilityEndDate,dependentEligibilityEndDate,dependentRelationshipTypeId,employerGroupCoverageStartDate,employerGroupCoverageEndDate,dependentEmployerGroupCoverageStartDate,dependentEmployerGroupCoverageEndDate,isVip,dependentCampaignSegmentGuid,dependentReportingValue1,dependentReportingValue2,dependentReportingValue3,dependentReportingValue4,dependentReportingValue5,honorific,dependentHonorific,hraAllocationScheduleEndDate,dependentHraAllocationScheduleEndDate,previousClientMemberId,dependentPreviousClientMemberId,divisionCode,dependentDivisionCode,mailingAddress1,mailingAddress2,mailingAddressCityName,mailingAddressStateCode,mailingAddressZipCode,mailingAddressCountryCode,nextAllocationScheduleAmount,nextAllocationScheduleFrequencyCode,nextAllocationScheduleStartDate,dateOfDeath,dateOfDivorce,dependentNextAllocationScheduleAmount,dependentNextAllocationScheduleFrequencyCode,dependentNextAllocationScheduleStartDate,dependentDateOfDeath,dependentDateOfDivorce,participantCode,dependentParticipantCode,errorCode,portalErrorCodes,personMatches,originalRowIndex
2013FormatTestFile.csv,2013FormatTestFile20191008_112620_OBFUSCATED.csv,2019-10-08 11:26:20 -06:00,100987757,James,S,CbjjJebjQA,19390226,M,20120701,N,,,,,,,,2,,,,,,,,,,,,,,,,,5053320000,5056810000,,,123 Quality Assurance St,,,ALBUQUERQUE,NM,87123,USA,Bernalillo,{A2468972-FEC0-41EC-9B8C-DBE04D6D1842},,,,,N,,,,,,,,,,,20151109,,,,,,,N,,,,,,,,,,,,,,,,,,,,,,,,20200604,,,,,,,,,,,,
2013FormatTestFile.csv,2013FormatTestFile20191008_112620_OBFUSCATED.csv,2019-10-08 11:26:20 -06:00,444986757,Kenneth,R,KaefKecjQA,19510817,M,20120701,N,,,20101101,,20101101,,100987757A,,129447898,PATRICIA,,KaefPcfbQA,19521017,F,20120701,N,,,,,,,,,3364710000,3368690000,testextend+qa@gmail.com,testextend+dependent+qa@gmail.com,123 Quality Assurance St,,,HIGH POINT,NC,27265,USA,Guilford,{9CA5D30C-13D2-40F7-B14C-1A2F0EAD15DD},,2000,A,20120701,N,,2000,A,20120701,N,,,,,,20160731,20170930,6,,,,,N,{9CA5D30C-13D2-40F7-B14C-1A2F0EAD15DD},,,,,,,,,,,,,,,,,,,,,O,,,,,O,,,,,,,,
"""), dtype='object')
test_data

,clientOriginalFileName,systemAssignedBaseFileName,originalFileProcessingStartedAtTime,socialSecurityNumber,firstName,middleName,lastName,dateOfBirth,genderCode,eligibilityStartDate,...,dependentNextAllocationScheduleFrequencyCode,dependentNextAllocationScheduleStartDate,dependentDateOfDeath,dependentDateOfDivorce,participantCode,dependentParticipantCode,errorCode,portalErrorCodes,personMatches,originalRowIndex
0,2013FormatTestFile.csv,2013FormatTestFile20191008_112620_OBFUSCATED.csv,2019-10-08 11:26:20 -06:00,100987757,James,S,CbjjJebjQA,19390226,M,20120701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013FormatTestFile.csv,2013FormatTestFile20191008_112620_OBFUSCATED.csv,2019-10-08 11:26:20 -06:00,444986757,Kenneth,R,KaefKecjQA,19510817,M,20120701,...,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
if row["dependentSocialSecurityNumber"] != "" 
or row["dependentClientMemberId"] != "" 
or row["dependentFirstName"] != "" 
or row["dependentLastName"] != ""
or row["dependentDateOfBirth"] != ""
or row["dependentCampaignSegmentGuid"] != "":
    print(row["dependentSocialSecurityNumber"])

In [12]:
for x in test_data.columns:
    print(x, test_data.at[0, x])

clientOriginalFileName 2013FormatTestFile.csv
systemAssignedBaseFileName 2013FormatTestFile20191008_112620_OBFUSCATED.csv
originalFileProcessingStartedAtTime 2019-10-08 11:26:20 -06:00
socialSecurityNumber 100987757
firstName James
middleName S
lastName CbjjJebjQA
dateOfBirth 19390226
genderCode M
eligibilityStartDate 20120701
eligibleIndicator N
clientSpecificPlanId nan
clientSpecificDentalId nan
medicarePartAStartDate nan
medicarePartAEndDate nan
medicarePartBStartDate nan
medicarePartBEndDate nan
healthInsuranceClaimNumber nan
eventCode 2
dependentSocialSecurityNumber nan
dependentFirstName nan
dependentMiddleName nan
dependentLastName nan
dependentDateOfBirth nan
dependentGenderCode nan
dependentEligibilityStartDate nan
dependentEligibleIndicator nan
dependentClientSpecificPlanId nan
dependentClientSpecificDentalId nan
dependentMedicarePartAStartDate nan
dependentMedicarePartAEndDate nan
dependentMedicarePartBStartDate nan
dependentMedicarePartBEndDate nan
dependentHealthInsuranceC

In [13]:
for x in test_data.columns:
    print(x, test_data.at[1, x])

clientOriginalFileName 2013FormatTestFile.csv
systemAssignedBaseFileName 2013FormatTestFile20191008_112620_OBFUSCATED.csv
originalFileProcessingStartedAtTime 2019-10-08 11:26:20 -06:00
socialSecurityNumber 444986757
firstName Kenneth
middleName R
lastName KaefKecjQA
dateOfBirth 19510817
genderCode M
eligibilityStartDate 20120701
eligibleIndicator N
clientSpecificPlanId nan
clientSpecificDentalId nan
medicarePartAStartDate 20101101
medicarePartAEndDate nan
medicarePartBStartDate 20101101
medicarePartBEndDate nan
healthInsuranceClaimNumber 100987757A
eventCode nan
dependentSocialSecurityNumber 129447898
dependentFirstName PATRICIA
dependentMiddleName nan
dependentLastName KaefPcfbQA
dependentDateOfBirth 19521017
dependentGenderCode F
dependentEligibilityStartDate 20120701
dependentEligibleIndicator N
dependentClientSpecificPlanId nan
dependentClientSpecificDentalId nan
dependentMedicarePartAStartDate nan
dependentMedicarePartAEndDate nan
dependentMedicarePartBStartDate nan
dependentMedic